In [1]:
import numpy as np 
import pandas as pd
import argparse, collections, configparser, io, json, math, os, requests, sys, time
from datetime import datetime
from requests import HTTPError
from requests import ConnectionError
from requests_oauthlib import OAuth1
from bs4 import BeautifulSoup

In [2]:
job = pd.read_csv('data/job.csv')
job.head()

,job_id,query
0,1,q=missouri
1,2,q=texas
2,3,q=california
3,4,q=london
4,5,q=france


In [3]:
consumer_key = '7StWFQlriMB0PWrp7MhrrYiid'
consumer_secret = '5FPZAdE7xCsO0jHzBBpgJOX5GeqQvhmGyUwFc8zX1iTlKflJDd'
access_token = '2808413395-7T0aDBeaVlGrgS5PJepbnybxWbTIdYd0ajn6KH4'
access_token_secret = 'EApjA21qAyzwTiO5lrk0ti9Zk6MYhJqsxCHz8nPHh1A2n'


oauth = OAuth1(client_key=consumer_key,
    client_secret=consumer_secret,
    resource_owner_key=access_token,
    resource_owner_secret=access_token_secret)

In [27]:
def search(query, oauth) :
    print("Query: " + query)
    
    attempt = 1
    while attempt <= 3 :
        try :
            r = requests.get("https://api.twitter.com/1.1/search/tweets.json?" + query+"&count=100", auth=oauth)
            return json.loads(r.text)

        except (ConnectionError, HTTPError) as err :
            print(err)
            sleep_time = 2**(attempt - 1)
            print("Connection attempt " + str(attempt) + " failed. "
                "Sleeping for " + str(sleep_time) + " second(s).")
            time.sleep(sleep_time)
            attempt = attempt + 1
    
    print("***** Error: Unable to query Twitter. Terminating.")

In [28]:
def getFullQuery(query) :
    if (not query.startswith("q=")) :
        query = "q=" + query
    return query

In [29]:
def structureTweetTable(tweet, job):
   
    list_o_dicts = []
    list_o_hashtags = []
    
    for i in tweet['statuses']:
        d = {}
        d['tweet_id_str'] =  i["id_str"] 
        d['job_id'] = job
        d['created_at'] = datetime.strptime(i["created_at"], '%a %b %d %H:%M:%S +0000 %Y').strftime('%Y-%m-%d %H:%M:%S')
        d['text'] = i["text"]
        source = cleantext = BeautifulSoup(i["source"]).text;
        d['source'] = source
        d['iso_language'] = i["metadata"]["iso_language_code"]
        for tag in i["entities"]["hashtags"]:
            h={}
            h['job_id'] =  job
            h['tweet_id_str'] =  i["id_str"]
            h['hashtag'] =  tag["text"]
            list_o_hashtags.append(h) 
        list_o_dicts.append(d)
    return (list_o_dicts,list_o_hashtags)

In [30]:
frames = []
hashtags_frame = []
for index, row in job.iterrows():
    
    job_id = row['job_id']
    q = getFullQuery(row['query'])
    results = search(q,oauth)
    try:
        data,tags = structureTweetTable(results,job_id)
        f = pd.DataFrame(data)
        frames.append(f)
        h = pd.DataFrame(tags)
        hashtags_frame.append(h)
    except Exception:
        print(e)
        print("   no results returned for '{}'".format(row['query']))
        pass

Query: q=missouri


/usr/lib/python3.4/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 170 of the file /usr/lib64/python3.4/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


Query: q=texas
Query: q=california
Query: q=london
Query: q=france


In [31]:
df = pd.concat(frames)
df_tags = pd.concat(hashtags_frame,ignore_index=True)

In [32]:
df.head(20)

,created_at,iso_language,job_id,source,text,tweet_id_str
0,2017-04-25 16:24:45,en,1,Twitter Web Client,#Redskins need to add to their pass rush. Miss...,856906838004178945
1,2017-04-25 16:24:21,en,1,Crowdfire - Go Big,Missouri Cowboy Poets Association event - Stor...,856906737718358016
2,2017-04-25 16:24:21,en,1,Twitter for iPhone,@bmark5472 @ChrisBragdon @SwolenBrain @ebell_m...,856906736208183296
3,2017-04-25 16:24:10,en,1,dlvr.it,"Lincoln University, State Tech still looking f...",856906692956520452
4,2017-04-25 16:24:10,en,1,Twitter for iPhone,RT @notmagicscience: TEACHERS: Working &amp; R...,856906692315025408
5,2017-04-25 16:24:06,en,1,Twitter for iPhone,RT @kwoodschools: US News and World Report ran...,856906675399380993
6,2017-04-25 16:23:58,en,1,Twitter for iPhone,RT @ValleyHush: KANSAS CITY MAY 5. Stoked to p...,856906641765273600
7,2017-04-25 16:23:53,en,1,Twitter for iPhone,@darlingmoon I'm working about 30 miles from M...,856906621594861569
8,2017-04-25 16:23:53,en,1,Twitter for Mac,RT @ValleyHush: KANSAS CITY MAY 5. Stoked to p...,856906619761942532
9,2017-04-25 16:23:47,en,1,Twitter for iPhone,RT @jrosenbaum: . @EricGreitens has appointed ...,856906594172489728


In [33]:
df_tags.head()

,hashtag,job_id,tweet_id_str
0,Redskins,1,856906838004178945
1,Missouri,1,856906416224862212
2,roleplay,1,856906416224862212
3,hotgirl,1,856906416224862212
4,love,1,856906416224862212


In [37]:
df_tags.to_csv('hashtags.csv', sep='|')


In [38]:
df.to_csv('tweets1.csv', sep='|')